# Random Ensemble Model

In [1]:
import sys
from os.path import abspath

sys.path.append(abspath('../src/'))

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from utils import *
from baseline_convnet import BaselineConvnet
import matplotlib.pyplot as plt
from sklearn import preprocessing
import random
from random_ensemble import RandomEnsemble

Using TensorFlow backend.


In [2]:
x_train, y_train, x_test, y_test, input_shape, num_classes = preprocess_mnist()
model = RandomEnsemble(input_shape=input_shape, num_classes=num_classes, n_proj=10, size_proj=8)


Loading mnist.
x_train shape: (60000, 28, 28, 1) 
x_test shape: (10000, 28, 28, 1)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [3]:
classifiers, projector = model.load_classifier(
    relative_path="../trained_models/baseline_convnet/baseline_proj10_size8/")

Instructions for updating:
Use tf.cast instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [4]:
model.evaluate_test(classifiers, x_test, y_test)


Testing infos:
x_test.shape =  (10000, 28, 28, 1) 
y_test.shape =  (10000, 10) 


Computing random projections.
Input shape:  (10000, 28, 28, 1)
Output shape: (10, 10000, 8, 8, 1) -> 10 random projections of 10000 images whose shape is (8, 8, 1)

Original test data.
Correctly classified: 9416
Incorrectly classified: 584
Test accuracy: 94.16%


In [11]:
x_test_fgsm = model.evaluate_adversaries(classifiers, x_test, y_test, method='fgsm')


Adversarial evaluation using FGSM method.

Computing random projections.
Input shape:  (10000, 28, 28, 1)
Output shape: (10, 10000, 8, 8, 1) -> 10 random projections of 10000 images whose shape is (8, 8, 1)

Adversarial test data.
Correctly classified: 4213
Incorrectly classified: 5787
Adversarial accuracy: 42.13%


In [4]:
x_test_adv = model.evaluate_adversaries(classifiers, x_test, y_test, method='deepfool', 
                                        adversaries_path='../data/mnist_x_test_deepfool.pkl')


Computing random projections.
Input shape:  (10000, 28, 28, 1)


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [5]:
x_test_carlini = model.evaluate_adversaries(classifiers, x_test, y_test, 
                                                           method='carlini_l2', 
                                                           adversaries_path='../data/mnist_x_test_carlini.pkl')


Computing random projections.
Input shape:  (10000, 28, 28, 1)
Output shape: (10, 10000, 8, 8, 1) -> 10 random projections of 10000 images whose shape is (8, 8, 1)

Adversarial test data.
Correctly classified: 9416
Incorrectly classified: 584
Adversarial accuracy: 94.16%


In [6]:
x_test_virtual = model.evaluate_adversaries(classifiers, x_test, y_test, 
                                                           method='virtual_adversarial', 
                                                           adversaries_path='../data/mnist_x_test_virtual.pkl')


Computing random projections.
Input shape:  (10000, 28, 28, 1)


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

## Adversarial training

In [7]:
classifiers, projector = model.load_classifier(
    relative_path="../trained_models/random_ensemble/robust_fgsm_proj10_size8/",
    model_name="random_ensemble_robust_fgsm")

In [8]:
x_test_pred_adv = model.evaluate_test(classifiers, x_test, y_test)


Testing infos:
x_test.shape =  (10000, 28, 28, 1) 
y_test.shape =  (10000, 10) 


Computing random projections.
Input shape:  (10000, 28, 28, 1)
Output shape: (10, 10000, 8, 8, 1) -> 10 random projections of 10000 images whose shape is (8, 8, 1)

Original test data.
Correctly classified: 9258
Incorrectly classified: 742
Test accuracy: 92.58%


In [8]:
x_test_fgsm, x_test_fsgm_pred = model.evaluate_adversaries(classifiers, x_test, y_test, method='fgsm')


Adversarial evaluation using FGSM method.

Computing random projections.
Input shape:  (10000, 28, 28, 1)
Output shape: (10, 10000, 8, 8, 1) -> 10 random projections of 10000 images whose shape is (8, 8, 1)

Adversarial test data.
Correctly classified: 7123
Incorrectly classified: 2877
Adversarial accuracy: 71.23%


In [9]:
x_test_adv, x_test_adv_pred = model.evaluate_adversaries(classifiers, x_test, y_test, method='deepfool', 
                                                         adversaries_path='../data/mnist_x_test_deepfool.pkl')


Computing random projections.
Input shape:  (10000, 28, 28, 1)


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [11]:
x_test_carlini, x_test_carlini_pred = model.evaluate_adversaries(classifiers, x_test, y_test, 
                                                           method='carlini_l2', 
                                                           adversaries_path='../data/mnist_x_test_carlini.pkl')


Computing random projections.
Input shape:  (10000, 28, 28, 1)
Output shape: (10, 10000, 8, 8, 1) -> 10 random projections of 10000 images whose shape is (8, 8, 1)

Adversarial test data.
Correctly classified: 9262
Incorrectly classified: 738
Adversarial accuracy: 92.62%


In [9]:
x_test_virtual, x_test_virtual_pred = model.evaluate_adversaries(classifiers, x_test, y_test, 
                                                                 method='virtual_adversarial',
                                                                 adversaries_path='../data/mnist_x_test_virtual.pkl')


Computing random projections.
Input shape:  (10000, 28, 28, 1)


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').